In [30]:
import os
from dotenv import load_dotenv

In [31]:
load_dotenv('./.env')
os.environ.get('PINECONE_ENV')

'gcp-starter'

* Instantiating the API_KEY

In [32]:
OPENAI_API_KEY = os.getenv('OPENAI_SECREY_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')

## LangChain getting started


In [33]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512, openai_api_key=OPENAI_API_KEY)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


* Asking AI a question.

In [34]:
output = llm('explique como funções python funcionam')

In [35]:
print(output)



Funções Python são blocos de código que executam uma tarefa específica quando são chamados. Elas são definidas usando a palavra-chave def. Uma função recebe argumentos e, opcionalmente, retorna um valor. Funções Python são usadas para modularizar a lógica do programa, tornando mais fácil de ler e debugar o código. Uma função também pode ser chamada repetidamente para executar a mesma tarefa várias vezes, tornando o código mais eficiente.


* Count Tokens

In [36]:
print(llm.get_num_tokens('explique como funções python funcionam'))

13


* list of questions and answers

In [37]:
output = llm.generate(['... é a capital da França',
                       'qual é a formula da área de um círculo?'])

In [38]:
print(output.generations)

[[Generation(text='?\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nA área de um círculo é dada por A = π.r², onde r é o raio do círculo.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [39]:
output.generations[0][0].text

'?\n\nParis.'

In [40]:
output = llm.generate(['escreva um slogan original para um restaurante de sushi'] * 3)

In [41]:
for o in output.generations:
    print(o[0].text, end='')



"Sushi com sabor de autenticidade: Venha sentir o verdadeiro sabor do Oriente!"

"A frescura do sabor japonês, no coração da cidade!"

"Sushi com sabores que vão te surpreender!"

## ChatModels: GPT-3.5-Turbo e GPT-4

In [1]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

- AIMESSAGE - Helps define the assistant's behavior.
- HUMANMESSAGE - What do we ask the assistant
- SYSTEMMESSAGE - Helps store previous responses.

In [2]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='Você é um especialista em machine learning que responde tudo em português.'),
    HumanMessage(content='explique em um parágrafo o que é machine learning.')
]
output = chat(messages)

In [3]:
print(output.content)

Machine learning, ou aprendizado de máquina, é um campo da inteligência artificial que se baseia na capacidade dos computadores de aprender e melhorar a partir de dados, sem serem explicitamente programados. Por meio de algoritmos e modelos estatísticos, o machine learning permite que as máquinas analisem grandes quantidades de dados e identifiquem padrões, tomando decisões e realizando tarefas de forma autônoma. Essa abordagem é amplamente utilizada em diversas áreas, como reconhecimento de padrões, previsão de resultados, classificação de dados e recomendação de produtos, entre outros.


## Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

In [6]:
template = """Você é um virologista experiente.
Escreva algumas frases sobre o sequinte {virus} em {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=template
)
print(prompt)

input_variables=['idioma', 'virus'] template='Você é um virologista experiente.\nEscreva algumas frases sobre o sequinte {virus} em {idioma}.'


In [7]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='HIV', idioma='Inglês'))
print(output)



1. HIV is a virus that can cause serious health complications if left untreated.
2. Early detection and treatment of HIV can help reduce the risk of further health complications.
3. Understanding how HIV is transmitted is important for preventing the spread of infection.
4. HIV is a manageable condition with the right combination of medications.
5. Effective HIV prevention strategies such as using condoms and practicing safe sex are essential for reducing the risk of transmission.
6. Educating yourself and your community about HIV can help reduce stigma and create an environment of acceptance.


## Simple Chains

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

template = """Você é um virologista experiente.
Escreva um resumo sobre o sequinte {virus} em {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HIV', 'idioma': 'english'})

In [9]:
output

'HIV, or Human Immunodeficiency Virus, is a viral infection that attacks the immune system, specifically the CD4 cells (T cells), which play a crucial role in fighting off infections. If left untreated, HIV can lead to the development of acquired immunodeficiency syndrome (AIDS).\n\nHIV is primarily transmitted through unprotected sexual intercourse, sharing contaminated needles, or from an infected mother to her child during childbirth or breastfeeding. It can also be transmitted through blood transfusions, though this is now rare due to improved screening techniques.\n\nUpon infection, individuals may experience flu-like symptoms, such as fever, fatigue, and swollen lymph nodes. However, these symptoms can be mild and easily overlooked. As the virus progresses, it weakens the immune system, making the individual more susceptible to various infections and diseases.\n\nThere is currently no cure for HIV, but it can be managed through antiretroviral therapy (ART). ART involves a combina

## Sequential Chains

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['conceito'],
    template="""Você é um cientista experiente e programador Python.
    Escreva uma função que implementa o conceito de {conceito}."""
)

chain1 = LLMChain(llm=llm1, prompt=prompt1)

#--- segunda chain ----

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template="""Dada a função {function} Python, descreva como funciona da forma mais detalhada possível."""
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run("softmax")



> Entering new SimpleSequentialChain chain...


def softmax(x): 
    # x is a vector/list of inputs 
    # compute the exponential of each element 
    exps = [np.exp(i) for i in x] 
    # compute the sum of all exponentials 
    sum_of_exps = np.sum(exps) 
    # compute the softmax 
    softmax = [j/sum_of_exps for j in exps] 
    # return the softmax 
    return softmax
A função softmax é uma função de ativação que é comumente usada na camada de saída de uma rede neural quando se trata de classificação multiclasse.
A função recebe um vetor ou lista de valores de entrada e realiza as seguintes etapas para calcular a saída softmax:

1. Para cada elemento de entrada x, a função calcula a exponencial de x usando a função np.exp(i), onde np é uma biblioteca de NumPy.
2. Esses valores exponenciais são armazenados em uma lista chamada "exps".
3. Em seguida, a função calcula a soma de todos os elementos da lista "exps" usando a função np.sum(exps) e armazena o resultado na variável "sum_of

## LangChain Agents

In [11]:
# exemplo exponenciação

5.1 ** 7.3

146306.05007233328

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonAstREPLTool(),
    verbose=True
)
agent_executor.run('qual a resposta para 5.1 elevado à potência de 7.3')